In [32]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/metall/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import numpy as np
import openpyxl
import pandas as pd


In [34]:
def parcer(path_to='/content/drive/My Drive/metall/Прогноз металлические заготовки/cloud/tables', path_from='/content/drive/My Drive/metall/metall.xlsx'):
  '''
  1) takes a table of the original format, removes all unnamed columns from it
  2) goes through the columns and divides the original table into a bunch of small ones (in the intervals between the date-columns)
  3) cleans the resulting mini-tables from empty tails and saves them to the specified location, naming the date-column's name
  4) returns a list of the names of the received tables, years of data start

  *) order of saving: old data on top. this is important when calling join_interpolate!
  '''
  excel_data=pd.read_excel(path_from, skiprows=[0,1])
  data = pd.DataFrame(excel_data)
  tables = []
  years = []
  current_table = pd.DataFrame()
  current_name = ''
  current_length = 0
  id_table = 0
  cols = data.columns
  for col in cols:
    if 'Unnamed' in col:
      data = data.drop([col], axis=1)
      continue
    if 'Date' in col:
      data[col] = pd.to_datetime(data[col], format='%Y-%m-%dT')
      if id_table == 0: 
        current_length = data[col].apply(lambda x: type(x) != type(pd.NaT)).sum()
        current_name = col
        id_table+=1
      else:
        current_table = current_table[current_table[current_name]<=start] # выровняем ВСЕ таблицы
        current_table=current_table.iloc[::-1]
        current_table.iloc[::-1].to_csv('{}/{}.csv'.format(path_to,current_name), index=False)
        tables.append('{}'.format(current_name))
        years.append(data[current_name].min().year)
        current_table = pd.DataFrame()
        current_length = data[col].apply(lambda x: type(x) != type(pd.NaT)).sum()
        current_name = col
        
    current_table[col] = data[col].head(current_length)
    current_table['year'] = data[current_name].apply(lambda x: x.year)
    current_table['month'] = data[current_name].apply(lambda x: x.month)
    current_table['day'] = data[current_name].apply(lambda x: x.day)
  tables.append('{}'.format(current_name))
  years.append(data[current_name].min().year)
  current_table=current_table.iloc[::-1]
  current_table = current_table[current_table[current_name]<=start]
  current_table.iloc[::-1].to_csv('{}/{}.csv'.format(path_to,current_name), index = False)
  return tables, years

Let's create many small tables of the usual format from the original large table

In [ ]:
start = pd.to_datetime('2021-04-02')
tables, years  = parcer()